In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn import metrics
from neupy import algorithms, estimators, environment, layers

Can not use cuDNN on context None: cannot compile with cuDNN. We got this error:
b'In file included from /tmp/try_flags_ydsvypwe.c:4:0:\n/usr/include/cudnn.h:63:26: fatal error: driver_types.h: No such file or directory\ncompilation terminated.\n'
Mapped name None to device cuda0: GeForce GTX 1070 (0000:01:00.0)


In [2]:
df = pd.read_csv("OnlineNewsPopularity.csv")

# Feature selection

In [3]:
headers = df.columns[0:61]
headers

Index(['url', ' timedelta', ' n_tokens_title', ' n_tokens_content',
       ' n_unique_tokens', ' n_non_stop_words', ' n_non_stop_unique_tokens',
       ' num_hrefs', ' num_self_hrefs', ' num_imgs', ' num_videos',
       ' average_token_length', ' num_keywords', ' data_channel_is_lifestyle',
       ' data_channel_is_entertainment', ' data_channel_is_bus',
       ' data_channel_is_socmed', ' data_channel_is_tech',
       ' data_channel_is_world', ' kw_min_min', ' kw_max_min', ' kw_avg_min',
       ' kw_min_max', ' kw_max_max', ' kw_avg_max', ' kw_min_avg',
       ' kw_max_avg', ' kw_avg_avg', ' self_reference_min_shares',
       ' self_reference_max_shares', ' self_reference_avg_sharess',
       ' weekday_is_monday', ' weekday_is_tuesday', ' weekday_is_wednesday',
       ' weekday_is_thursday', ' weekday_is_friday', ' weekday_is_saturday',
       ' weekday_is_sunday', ' is_weekend', ' LDA_00', ' LDA_01', ' LDA_02',
       ' LDA_03', ' LDA_04', ' global_subjectivity',
       ' global_sent

In [4]:
#remove recent news (less than 2 months)
df = df[df[' timedelta'] > 60]
data=df[df.columns[2:60]]
target = df[' shares'].ravel()

In [5]:
data_norm = StandardScaler().fit_transform(data)
data_norm.shape

(36434, 58)

In [7]:
import statsmodels.api as sm

/home/milton/anaconda3/envs/tensorflow/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [8]:
x_train, x_test, y_train, y_test = train_test_split(data_norm, target, test_size=0.3)
x_train_new = sm.add_constant(x_train)
x_test_new = sm.add_constant(x_test)
full_mod = sm.OLS(y_train,x_train_new)


In [9]:
full_res = full_mod.fit()
print ("\n \n",full_res.summary())


 
                             OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.024
Model:                            OLS   Adj. R-squared:                  0.022
Method:                 Least Squares   F-statistic:                     11.16
Date:                Wed, 02 May 2018   Prob (F-statistic):           9.42e-96
Time:                        23:46:08   Log-Likelihood:            -2.7527e+05
No. Observations:               25503   AIC:                         5.507e+05
Df Residuals:                   25446   BIC:                         5.511e+05
Df Model:                          56                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1.025e+10   5.16e+09     -1.988     


Variance Inflation Factor


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [15]:
from sklearn.decomposition import PCA

#pca = PCA(n_components=58).fit(data_norm)
#pca_samples = pca.transform(data_norm)

pca = PCA(n_components=22, whiten=True)
X_pca = pca.fit_transform(data_norm)

In [13]:
import matplotlib.pyplot as plt
def pca_results(data, pca):
    
    # Dimension indexing
    dimensions = ['Dimension {}'.format(i) for i in range(1,len(pca.components_)+1)]
    
    # PCA components
    components = pd.DataFrame(np.round(pca.components_, 4), columns = data.keys()) 
    components.index = dimensions

    # PCA explained variance
    ratios = pca.explained_variance_ratio_.reshape(len(pca.components_), 1) 
    variance_ratios = pd.DataFrame(np.round(ratios, 4), columns = ['Explained Variance']) 
    variance_ratios.index = dimensions

    # Create a bar plot visualization
    fig, ax = plt.subplots(figsize = (14,8))

    # Plot the feature weights as a function of the components
    components.plot(ax = ax, kind = 'bar')
    ax.set_ylabel("Feature Weights") 
    ax.set_xticklabels(dimensions, rotation=0)

    # Display the explained variance ratios# 
    for i, ev in enumerate(pca.explained_variance_ratio_): 
        ax.text(i-0.40, ax.get_ylim()[1] + 0.05, "Explained Variance\n %.4f"%(ev))

    # Return a concatenated DataFrame
    return pd.concat([variance_ratios, components], axis = 1)

pca_results = pca_results(data, pca)

array([[ 0.30329888, -0.52104412,  0.03261249, ...,  0.20150741,
        -0.73642939,  0.43632472],
       [-0.50140898, -0.4363532 ,  0.02323239, ..., -0.30674946,
         0.4757911 , -0.90066635],
       [-0.4762814 , -0.26704636, -0.02683119, ..., -0.29224777,
         0.57605171, -0.77678163],
       ..., 
       [-1.71928696,  0.10995177, -0.02845738, ...,  1.15145243,
        -0.61764181,  1.5352717 ],
       [ 1.33793751,  0.78594727,  0.01109135, ..., -0.50049545,
        -0.80682667,  0.42244596],
       [ 0.04637521, -0.24280956, -0.00557315, ..., -0.35993354,
         0.177466  , -0.44632924]])

In [17]:
X_rebuilt.shape

(36434, 58)

In [14]:
pca_results.cumsum()

,Explained Variance,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,...,avg_positive_polarity,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity
Dimension 1,0.0788,-0.0299,0.1349,0.0001,0.0028,0.0021,0.1712,0.1311,0.0959,0.0660,...,0.3371,0.0627,3.435000e-01,-0.1642,-0.1363,-0.0971,0.0914,0.0995,-0.0539,0.1039
Dimension 2,0.1496,0.0359,0.2083,-0.0002,0.0035,0.0013,0.2755,0.1344,0.2042,0.1989,...,0.3801,0.1004,3.710000e-01,-0.4292,-0.4233,-0.1417,0.1691,0.0274,-0.0714,0.1634
Dimension 3,0.2096,0.0531,0.0381,-0.0024,-0.0033,-0.0039,0.2084,0.0714,0.1900,0.2139,...,0.2855,0.0965,2.422000e-01,-0.2611,-0.1841,-0.1457,0.2350,0.1200,-0.1310,0.2366
Dimension 4,0.2614,0.0508,0.0393,0.5743,0.5734,0.5728,0.2072,0.0788,0.2069,0.2125,...,0.2815,0.0996,2.345000e-01,-0.2565,-0.1753,-0.1482,0.2278,0.1167,-0.1355,0.2293
Dimension 5,0.3085,0.0046,0.0928,0.5884,0.5893,0.5864,0.2734,0.1566,0.2872,0.1695,...,0.2322,0.0883,1.998000e-01,-0.2101,-0.1756,-0.0785,0.1952,0.0968,-0.0940,0.1978
Dimension 6,0.3527,-0.0788,0.1225,0.5827,0.5853,0.5832,0.2863,0.1748,0.1932,0.0987,...,0.2775,0.1025,2.379000e-01,-0.2509,-0.2225,-0.1207,-0.1099,-0.0916,0.1612,-0.1085
Dimension 7,0.3948,-0.1364,-0.1334,0.5884,0.5844,0.5897,0.0642,-0.0570,-0.0279,0.1060,...,0.3570,0.2665,1.884000e-01,-0.3797,-0.2140,-0.3075,-0.1556,-0.1661,0.2149,-0.1619
Dimension 8,0.4344,-0.1670,0.0990,0.5881,0.5899,0.5906,0.2428,0.0252,0.0384,0.0736,...,0.3280,0.0884,2.931000e-01,-0.3285,-0.2900,-0.1723,-0.2749,-0.2009,0.3101,-0.2669
Dimension 9,0.4710,-0.1469,0.2101,0.5931,0.5976,0.5967,0.2802,-0.0048,-0.0229,0.0549,...,0.2184,-0.1160,2.959000e-01,-0.3084,-0.3554,-0.0876,0.0197,0.0065,0.0367,0.0403
Dimension 10,0.5062,-0.0647,0.0576,0.5992,0.6007,0.6035,0.1652,-0.1745,-0.1521,-0.0142,...,0.3587,0.0892,3.121000e-01,-0.3167,-0.2899,-0.1965,0.2636,0.1654,-0.1500,0.2866


In [ ]:
#Explained variance
plt.plot(pca.explained_variance_ratio_)
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')
plt.show()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data_norm, target, test_size=0.3)

In [ ]:
grnnet = algorithms.GRNN(std=0.5, verbose=True)
grnnet.train(x_train, y_train)

In [ ]:
result = grnnet.predict(x_test)

In [ ]:
if np.isnan(result).any():
    clean = np.nan_to_num(result)

In [ ]:
error = estimators.rmse(clean, y_test)
error

In [ ]:
r2_score = metrics.r2_score(y_test, clean)
r2_score

In [ ]:
mae = metrics.median_absolute_error(y_test, clean)
mae